# Manipulação de gráficos em planilhas Excel com o Python
-----------

O objetivo desse notebook é compreender como podemos manipular gráficos em Excel para facilitar a criação de reports e dashs diretamente do Python, sem a necessidade de baixar um Pandas DataFrame para um Excel e criar dentro do software todos os gráficos desejados.

In [28]:
#Importando bibliotecas
import numpy as np
import pandas as pd
import openpyxl

In [44]:
#Abrindo a base de diamantes do Kaggle
df = pd.read_csv('largest_diamond_dataset_kaggle_1604.csv', index_col='Unnamed: 0')
df.head()

,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,culet_size,...,meas_depth,girdle_min,girdle_max,fluor_color,fluor_intensity,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,total_sales_price
0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,N,...,1.79,M,M,unknown,None,unknown,unknown,unknown,unknown,200
1,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,N,...,1.78,STK,STK,unknown,None,unknown,unknown,unknown,unknown,200
2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,unknown,...,1.77,TN,M,unknown,None,unknown,unknown,unknown,unknown,200
3,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,unknown,...,1.78,M,STK,unknown,None,unknown,unknown,unknown,unknown,200
4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,N,...,1.82,STK,STK,unknown,None,unknown,unknown,unknown,unknown,200


O openpyxl tem grande ligação com o Pandas assim, este oferece um método para facilmente passar de um objeto Pandas DataFrame para um objeto de worksheet dentro do Python.

In [45]:
#DataFrame -> Worksheet
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
wb = Workbook()
ws = wb.active

#for r in dataframe_to_rows(df, index=True, header=True):
#    ws.append(r)

In [46]:
#Criando uma guia para cálculos baseado na guia 1
ws_backend = wb.create_sheet('backend')

In [47]:
wb.sheetnames

['Sheet', 'backend']

In [48]:
#Criando DataFrame de COUNT.IFS. da coluna 'cut_quality'
for row in dataframe_to_rows(df['cut_quality'].value_counts().to_frame().reset_index(drop=False), index=False, header=False):
    ws_backend.append(row)

In [49]:
#Criando gráfico de barras com o COUNT da coluna 'cut_quality'
from openpyxl.chart import BarChart, Reference, Series
#Defina os valores que serão imputados
data_cut_quality = Reference(ws_backend, min_col=2, min_row=1, max_col=2, max_row=6)
categories_cut_quality = Reference(ws_backend, min_col=1, min_row=1, max_col=1, max_row=6)

#Crie o objeto do gráfico
grafico_barra_cut_quality = BarChart()

#Defina os atributos do objeto
grafico_barra_cut_quality.type = 'col'
grafico_barra_cut_quality.title = 'Distribuição de diamantes por qualidade da lapidação'
grafico_barra_cut_quality.x_axis.title = 'Qualidade'
grafico_barra_cut_quality.y_axis.title = 'qtd. diamantes'
grafico_barra_cut_quality.legend = None

#Adicione os dados e as categorias
grafico_barra_cut_quality.add_data(data_cut_quality)
grafico_barra_cut_quality.set_categories(categories_cut_quality)

#Adicione o objeto do gráfico ao objeto da worksheet
ws_backend.add_chart(grafico_barra_cut_quality, anchor='G3')

In [50]:
#Inserindo as colunas de meas_depth e total_sales_price para um scatter plot
#Define primeira linha que iremos inserir os dados
linha_excel = 2
coluna_excel = 'D'
#Define o nome da coluna no Excel
ws_backend[f'{coluna_excel}1'] = df.loc[:, 'meas_depth'].name
#Itera para inserir os dados na sheet
for row in df.loc[:, 'meas_depth'].index:
    #Passa para o Excel o valor da linha
    ws_backend[f'{coluna_excel}{linha_excel}'] = df.loc[row, 'meas_depth']
    #Soma 1 para ir para a linha debaixo
    linha_excel += 1

#total_sales_price
linha_excel_2 = 2
coluna_excel_2 = 'E'
#Define o nome da coluna no Excel
ws_backend[f'{coluna_excel_2}1'] = df.loc[:, 'total_sales_price'].name
#Itera para inserir os dados na sheet
for row in df.loc[:, 'total_sales_price'].index:
    #Passa para o Excel o valor da linha
    ws_backend[f'{coluna_excel_2}{linha_excel_2}'] = df.loc[row, 'total_sales_price']
    #Soma 1 para ir para a linha debaixo
    linha_excel_2 += 1

#Criando o Scatterplot -----------------------------------------------------------------
from openpyxl.chart import ScatterChart
#Cria o objeto do scatterplot
scatterplot_meas_sales = ScatterChart()

#Define os atributos do Scatterplot
scatterplot_meas_sales.title = 'Relação Meas Depth x Total Sales Price'
scatterplot_meas_sales.x_axis.title = 'Meas Depth'
scatterplot_meas_sales.y_axis.title = 'Total Sales Price'
scatterplot_meas_sales.legend = None

#Criando as series de dados
xvalues = Reference(ws_backend, min_col=4, min_row=2, max_col=4, max_row=ws_backend.max_row)
yvalues = Reference(ws_backend, min_col=5, min_row=2, max_col=5, max_row=ws_backend.max_row)
series_dados = Series(yvalues, xvalues, title_from_data=True)
series_dados.marker = openpyxl.chart.marker.Marker('circle')
series_dados.graphicalProperties.line.noFill=True
scatterplot_meas_sales.series.append(series_dados)

#Adicionando ao objeto da sheet o gráfico
ws_backend.add_chart(scatterplot_meas_sales, anchor='Q3')

In [51]:
wb.save('Teste_Graficos.xlsx')

Próximo desafio é criar uma função que automaticamente consiga inserir os dados na planilha e gerar o gráfico solicitado pelo usuário com as seguintes especificações:

- Deve procurar a primeira célula vazia mais à direita da planilha e inserir os dados ali automaticamente;
- Só recebe DataFrames no formato de dados do gráfico desejado;
- Deve receber qualquer gráfico entre Linha, Barras, Dispersão e Combinação de Linhas/Barras;
- Pode ter os atributos do gráfico todos definidos;
- Deve solicitar qual a planilha que os dados e o gráfico serão inseridos;
- Tetnar colocar o gráfico sempre onde não há 1 gráfico já.

In [53]:
def criar_graficos_em_planilhas_excel():
    pass